In [5]:
from collections import defaultdict
from dataclasses import dataclass, field
from typing import List, Set, Tuple, Dict
import heapq
from enum import Enum

In [ ]:
class Direction(Enum):
    FORWARD = (0, 1)  # Initially facing east
    RIGHT = (1, 0)    # 90° clockwise
    BACKWARD = (0, -1) # 180° turn
    LEFT = (-1, 0)    # 90° counterclockwise

    def turn_left(self):
        return {
            Direction.FORWARD: Direction.LEFT,
            Direction.LEFT: Direction.BACKWARD,
            Direction.BACKWARD: Direction.RIGHT,
            Direction.RIGHT: Direction.FORWARD
            }[self]

    def turn_right(self):
        return {
            Direction.FORWARD: Direction.RIGHT,
            Direction.RIGHT: Direction.BACKWARD,
            Direction.BACKWARD: Direction.LEFT,
            Direction.LEFT: Direction.FORWARD
        }[self]

In [7]:
@dataclass(frozen=True, order=True)
class State:
    # Add a compare_key field that will be used for ordering
    compare_key: tuple = field(init=False, repr=False)
    row: int
    col: int
    direction: Direction

    def __post_init__(self):
        # Create a tuple for comparison
        object.__setattr__(self, 'compare_key', (self.row, self.col, self.direction.value))

In [ ]:
def solve_maze(maze: List[str]) -> int:
    # Find start and end positions
    start_pos = end_pos = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'S':
                start_pos = (i, j)
            elif cell == 'E':
                end_pos = (i, j)

    # Initialize distances dictionary and priority queue
    distances = {}
    start_state = State(start_pos[0], start_pos[1], Direction.FORWARD)
    distances[start_state] = 0
    pq = [(0, start_state)]
    seen = set()

    while pq:
        current_cost, current_state = heapq.heappop(pq)

        if (current_state.row, current_state.col) == end_pos:
            return current_cost

        if current_state in seen:
            continue

        seen.add(current_state)

        # Try moving forward
        dr, dc = current_state.direction.value
        new_row, new_col = current_state.row + dr, current_state.col + dc

        if (0 <= new_row < len(maze) and
            0 <= new_col < len(maze[0]) and
            maze[new_row][new_col] != '#'):
            new_state = State(new_row, new_col, current_state.direction)
            new_cost = current_cost + 1
            if new_state not in distances or new_cost < distances[new_state]:
                distances[new_state] = new_cost
                heapq.heappush(pq, (new_cost, new_state))

        # Try turning left/right
        for new_direction in [current_state.direction.turn_left(),
                            current_state.direction.turn_right()]:
            new_state = State(current_state.row, current_state.col, new_direction)
            new_cost = current_cost + 1000
            if new_state not in distances or new_cost < distances[new_state]:
                distances[new_state] = new_cost
                heapq.heappush(pq, (new_cost, new_state))

    return float('inf')  # No path found

In [14]:
file = "example-mini"
# file = "example"
# file = "input"

In [15]:
with open(file) as f:
    maze = f.read().strip().split('\n')

In [16]:
solve_maze(maze)

7036

In [30]:
def find_all_possible_paths(maze: List[str]) -> List[List[Tuple[int, int]]]:
    def dfs(pos: Tuple[int, int], direction: Direction,
           current_path: List[Tuple[int, int]],
           turns_at_pos: Dict[Tuple[int, int], Set[Direction]],
           all_paths: List[List[Tuple[int, int]]]) -> None:

        row, col = pos

        # Found end
        if maze[row][col] == 'E':
            all_paths.append(current_path[:])
            return

        # Try moving forward first
        dr, dc = direction.value
        new_row, new_col = row + dr, col + dc

        if (0 <= new_row < len(maze) and
            0 <= new_col < len(maze[0]) and
            maze[new_row][new_col] != '#' and
            (new_row, new_col) not in current_path):
            current_path.append((new_row, new_col))
            dfs((new_row, new_col), direction, current_path, turns_at_pos, all_paths)
            current_path.pop()

        # Try turning only if we can't move forward or haven't tried this direction
        else:
            for new_direction in [direction.turn_left(), direction.turn_right()]:
                # Check if we've already tried this direction at this position
                if pos not in turns_at_pos:
                    turns_at_pos[pos] = set()
                if new_direction not in turns_at_pos[pos]:
                    turns_at_pos[pos].add(new_direction)
                    dfs(pos, new_direction, current_path, turns_at_pos, all_paths)
                    turns_at_pos[pos].remove(new_direction)

    # Find start position
    start_pos = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'S':
                start_pos = (i, j)
                break
        if start_pos:
            break

    all_paths = []
    turns_at_pos = {}
    dfs(start_pos, Direction.FORWARD, [start_pos], turns_at_pos, all_paths)
    return all_paths

In [31]:
find_all_possible_paths(maze)

[]

In [25]:
len(union)

37

In [32]:
from dataclasses import dataclass
from heapq import heappop, heappush

from sys import maxsize


@dataclass(eq=True, frozen=True)
class State:
    cost: int
    position: complex
    orientation: complex

    def __lt__(self, other):
        return self.cost < other.cost


def day16(filename):
    print()
    print(filename)

    part1 = maxsize
    part2 = 0

    paths = set()
    with open(filename) as puzzle:
        for row, line in enumerate(puzzle):
            for col, cell in enumerate(line.strip()):
                if cell == "#":
                    continue

                pos = complex(row, col)
                paths.add(pos)
                if cell == "S":
                    start = pos
                elif cell == "E":
                    end = pos

    start_state = State(0, start, 1j)
    states = [(start_state, [start_state])]
    end_states = set()

    best_cost = {(start, 1j): 0}

    while states:
        state, path = heappop(states)
        if state.position == end:
            if state.cost <= part1:
                part1 = state.cost
                end_states.add((state, tuple(path)))

            continue

        for d in (1, 1j, -1j):
            direction = state.orientation * d
            look = state.position + direction
            if look in paths:
                cost = state.cost + 1 + 1000 * (state.orientation != direction)
                new_state = State(cost, look, direction)
                if best_cost.get((new_state.position, direction), maxsize) < cost:
                    continue

                heappush(states, (new_state, path + [new_state]))
                best_cost[(look, direction)] = cost

    best_seats = set.union(*[{s.position for s in path} for _, path in end_states])
    part2 = len(best_seats)
    print("part1", part1)
    print("part2", part2)
    return part1, part2


In [34]:
day16("input")


input
part1 74392
part2 426


(74392, 426)